In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Smart Personal Task & Mail Assistant (Concierge Track)

**Capstone:** Demonstrates Tool Calling, Memory, and Multi-step Reasoning using ADK patterns  
**Author:** Rasala Srinivas  
**Track:** Concierge Agents  
**Submission:** Kaggle Notebook + GitHub Repo  


In [2]:
# Kaggle kernel usually has most libs; install if needed
!pip install -q python-dateutil pytz imageio

In [3]:
# Creates project files (src/, demo_data/, requirements.txt, README.md)
import os, textwrap, json

PROJECT_ROOT = "/kaggle/working/smart-concierge-agent"
os.makedirs(PROJECT_ROOT, exist_ok=True)

files = {
 "requirements.txt": textwrap.dedent("""\
    pandas==2.2.2
    python-dateutil==2.8.2
    pytz==2024.1
    imageio==2.31.1
 """),
 "README.md": textwrap.dedent("""\
    # Smart Personal Task & Mail Assistant
    Concierge-track capstone demonstrating tool-calls, memory and multi-step reasoning.

    See notebook for usage.
 """),
 "demo_data/sample_emails.json": json.dumps([
  {
    "from":"alice@example.com",
    "subject":"Can we schedule a quick sync next week?",
    "body":"Hi, can we schedule a quick 30 min sync to discuss the Q4 roadmap? I'm available Thursday and Friday afternoons. Please let me know what works for you. Thanks!"
  },
  {
    "from":"bob@example.com",
    "subject":"Please prepare the monthly sales report",
    "body":"Hi, could you prepare the monthly sales report and share by Monday? Also add the revenue numbers for each region. Thanks!"
  }
 ], indent=2),
 "src/utils.py": textwrap.dedent("""\
    from dateutil import parser
    from datetime import datetime
    import json, os, pytz

    def load_json(path):
        with open(path, "r", encoding="utf-8") as f:
            return json.load(f)

    def save_json(path, obj):
        dirn = os.path.dirname(path)
        if dirn and not os.path.exists(dirn):
            os.makedirs(dirn)
        with open(path, "w", encoding="utf-8") as f:
            json.dump(obj, f, indent=2, ensure_ascii=False)

    def parse_date(text, tz="UTC"):
        try:
            dt = parser.parse(text, fuzzy=True)
            return dt.astimezone(pytz.timezone(tz))
        except Exception:
            return None
 """),
 "src/memory.py": textwrap.dedent("""\
    from pathlib import Path
    import json

    class MemoryStore:
        def __init__(self, path='memory.json'):
            self.path = Path(path)
            if not self.path.exists():
                self._write({})
        def _read(self):
            return json.loads(self.path.read_text(encoding='utf-8'))
        def _write(self,obj):
            self.path.write_text(json.dumps(obj, indent=2, ensure_ascii=False), encoding='utf-8')
        def get(self,key,default=None):
            data = self._read()
            return data.get(key, default)
        def set(self,key,value):
            data = self._read()
            data[key] = value
            self._write(data)
        def append_task(self, task):
            data = self._read()
            tasks = data.get('tasks', [])
            tasks.append(task)
            data['tasks'] = tasks
            self._write(data)
 """),
 "src/tools/email_tool.py": textwrap.dedent("""\
    import re

    def parse_email(email_obj: dict):
        body = email_obj.get('body','')
        subject = email_obj.get('subject','')
        sender = email_obj.get('from','')
        actions = []
        if re.search(r'\\b(schedule|meet|appointment|call|zoom|meeting)\\b', body, re.I):
            actions.append('schedule_meeting')
        if re.search(r'\\b(todo|task|please do|could you|please)\\b', body, re.I):
            actions.append('create_task')
        summary = body.strip().replace('\\n',' ')[:800]
        return {
            'from': sender,
            'subject': subject,
            'summary': summary,
            'actions': actions,
            'raw': body
        }

    def summarize_for_user(parsed):
        return f\"Email from {parsed['from']}: {parsed['summary'][:240]}\"
 """),
 "src/tools/calendar_tool.py": textwrap.dedent("""\
    from datetime import datetime, timedelta
    import uuid, json, os

    def suggest_event(parsed_email):
        now = datetime.utcnow()
        start = now + timedelta(days=1)
        start = start.replace(hour=10, minute=0, second=0, microsecond=0)
        end = start + timedelta(hours=1)
        event = {
            'id': str(uuid.uuid4()),
            'title': parsed_email.get('subject','Meeting'),
            'start': start.isoformat()+'Z',
            'end': end.isoformat()+'Z',
            'attendees': [parsed_email.get('from')]
        }
        return event

    def create_event(event, storage_path='demo_calendar.json'):
        cal = []
        if os.path.exists(storage_path):
            try:
                with open(storage_path,'r',encoding='utf-8') as f:
                    cal = json.load(f)
            except:
                cal = []
        cal.append(event)
        with open(storage_path,'w',encoding='utf-8') as f:
            json.dump(cal, f, indent=2, ensure_ascii=False)
        return event
 """),
 "src/tools/task_tool.py": textwrap.dedent("""\
    from src.memory import MemoryStore
    from datetime import datetime

    def create_task(parsed_email, priority='medium'):
        task = {
            'title': parsed_email.get('subject','Task'),
            'desc': parsed_email.get('summary'),
            'priority': priority,
            'created_at': datetime.utcnow().isoformat()+'Z',
            'source': 'email'
        }
        mem = MemoryStore()
        mem.append_task(task)
        return task
 """),
 "src/agent.py": textwrap.dedent("""\
    from src.tools.email_tool import parse_email, summarize_for_user
    from src.tools.calendar_tool import suggest_event, create_event
    from src.tools.task_tool import create_task
    from src.memory import MemoryStore

    class ConciergeAgent:
        def __init__(self, memory_path='memory.json'):
            self.memory = MemoryStore(memory_path)
        def handle_email(self, email_obj):
            parsed = parse_email(email_obj)
            actions = parsed.get('actions', [])
            result = {'parsed': parsed, 'decisions': []}
            if 'schedule_meeting' in actions:
                ev = suggest_event(parsed)
                created = create_event(ev)
                result['decisions'].append({'action':'create_event','event':created})
            if 'create_task' in actions or 'todo' in parsed.get('raw','').lower():
                task = create_task(parsed, priority='high')
                result['decisions'].append({'action':'create_task','task':task})
            reply = self._draft_reply(parsed)
            result['decisions'].append({'action':'draft_reply','reply':reply})
            return result

        def _draft_reply(self, parsed):
            summary = summarize_for_user(parsed)
            reply = f\"Hi {parsed.get('from')},\\n\\nThanks for your message. Quick summary: {summary}\\n\\nI can: 1) Schedule a meeting 2) Create a task for this.\\nPlease confirm which you'd prefer or let me proceed.\\n\\nBest,\\nSmart Assistant\"
            return reply
 """)
}

for path, content in files.items():
    full = os.path.join(PROJECT_ROOT, path)
    dirn = os.path.dirname(full)
    if dirn and not os.path.exists(dirn):
        os.makedirs(dirn, exist_ok=True)
    with open(full, "w", encoding="utf-8") as f:
        f.write(content)

print("Project files written to", PROJECT_ROOT)


Project files written to /kaggle/working/smart-concierge-agent


In [4]:
import sys
sys.path.append("/kaggle/working/smart-concierge-agent")

from src.agent import ConciergeAgent
from src.utils import load_json
from pathlib import Path
import json, os

emails = load_json("/kaggle/working/smart-concierge-agent/demo_data/sample_emails.json")
agent = ConciergeAgent(memory_path="/kaggle/working/smart-concierge-agent/memory.json")

results = []
for e in emails:
    print("=== Input ===")
    print("From:", e["from"])
    print("Subject:", e["subject"])
    r = agent.handle_email(e)
    results.append(r)
    print("\nDecisions:")
    for d in r["decisions"]:
        if d["action"]=="draft_reply":
            print("- draft_reply (preview):")
            print(d["reply"][:400])
        else:
            print("-", d["action"], ":", json.dumps(d.get(list(d.keys())[1]), indent=2) if isinstance(d.get(list(d.keys())[1]), dict) else d.get(list(d.keys())[1]))
    print("\n----\n")

# Show stored tasks and calendar (files)
mem_path = "/kaggle/working/smart-concierge-agent/memory.json"
cal_path = "demo_calendar.json"
print("Memory file path:", mem_path)
if os.path.exists(mem_path):
    print("Memory contents:", json.loads(open(mem_path).read()))
else:
    print("No memory found.")

if os.path.exists(cal_path):
    print("Calendar file (demo_calendar.json):")
    print(json.loads(open(cal_path).read()))
else:
    print("No calendar events created yet.")


=== Input ===
From: alice@example.com
Subject: Can we schedule a quick sync next week?

Decisions:
- create_event : {
  "id": "f23c1c08-f6b0-477e-8668-44a121eb4748",
  "title": "Can we schedule a quick sync next week?",
  "start": "2025-11-20T10:00:00Z",
  "end": "2025-11-20T11:00:00Z",
  "attendees": [
    "alice@example.com"
  ]
}
- create_task : {
  "title": "Can we schedule a quick sync next week?",
  "desc": "Hi, can we schedule a quick 30 min sync to discuss the Q4 roadmap? I'm available Thursday and Friday afternoons. Please let me know what works for you. Thanks!",
  "priority": "high",
  "created_at": "2025-11-19T09:21:01.351429Z",
  "source": "email"
}
- draft_reply (preview):
Hi alice@example.com,

Thanks for your message. Quick summary: Email from alice@example.com: Hi, can we schedule a quick 30 min sync to discuss the Q4 roadmap? I'm available Thursday and Friday afternoons. Please let me know what works for you. Thanks!

I can: 1) Schedule a meeting 2) Create a task for 

In [5]:
import json, pandas as pd
mem = json.loads(open("/kaggle/working/smart-concierge-agent/memory.json").read())
tasks = mem.get("tasks", [])
print("Stored tasks count:", len(tasks))
if tasks:
    display(pd.DataFrame(tasks))
if os.path.exists("demo_calendar.json"):
    cal = json.loads(open("demo_calendar.json").read())
    print("Calendar events:")
    display(pd.DataFrame(cal))


Stored tasks count: 0
Calendar events:


,id,title,start,end,attendees
0,f23c1c08-f6b0-477e-8668-44a121eb4748,Can we schedule a quick sync next week?,2025-11-20T10:00:00Z,2025-11-20T11:00:00Z,[alice@example.com]


In [6]:
# This is a simple automated GIF: renders the reply drafts as images then joins them.
from PIL import Image, ImageDraw, ImageFont
import imageio, os, json, textwrap

OUT_DIR = "/kaggle/working/smart-concierge-agent/gif_frames"
os.makedirs(OUT_DIR, exist_ok=True)

# get reply drafts
results_path = "/kaggle/working/smart-concierge-agent/memory.json"
with open("/kaggle/working/smart-concierge-agent/demo_data/sample_emails.json") as f:
    sample_emails = json.load(f)

from src.agent import ConciergeAgent
agent = ConciergeAgent(memory_path="/kaggle/working/smart-concierge-agent/memory.json")
frames = []
i = 0
for e in sample_emails:
    r = agent.handle_email(e)
    reply = [d['reply'] for d in r['decisions'] if d['action']=='draft_reply'][0]
    # render text to image
    W, H = 900, 300
    img = Image.new('RGB', (W,H), color='white')
    d = ImageDraw.Draw(img)
    try:
        font = ImageFont.truetype("DejaVuSans.ttf", 14)
    except:
        font = ImageFont.load_default()
    lines = textwrap.wrap(reply, width=120)
    y = 10
    for line in lines:
        d.text((10,y), line, fill=(0,0,0), font=font)
        y += 18
    frame_path = os.path.join(OUT_DIR, f"frame_{i}.png")
    img.save(frame_path)
    frames.append(frame_path)
    i += 1

# join into gif
imgs = [imageio.imread(f) for f in frames]
gif_path = "/kaggle/working/smart-concierge-agent/demo_reply.gif"
imageio.mimsave(gif_path, imgs, duration=2)
print("GIF created at:", gif_path)


/tmp/ipykernel_13/93824186.py:39: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  imgs = [imageio.imread(f) for f in frames]


GIF created at: /kaggle/working/smart-concierge-agent/demo_reply.gif


## Submission Checklist
- [ ] Notebook shows sample input → agent decisions → created calendar events and tasks.
- [ ] `demo_data/sample_emails.json` included.
- [ ] `memory.json` or demonstration of memory usage present.
- [ ] GIF demo included (`demo_reply.gif`).
- [ ] README.md in repo explaining how to run.
- [ ] GitHub repo link included in Kaggle submission form.
